In [21]:
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.emulators import *
from autoemulate.experimental.simulations.flow_problem import FlowProblem
from autoemulate.experimental.simulations.projectile import Projectile

In [2]:
param_ranges = {"T": (0.5, 2.0),
                "td": (0.1, 0.5),
                "amp": (100.0, 1000.0),
                "dt": (0.0001, 0.01),
                "ncycles": (5, 20),
                "ncomp": (5, 20),
                "C": (20.0, 60.0),
                "R": (0.01, 0.1),
                "L": (0.001, 0.005),
                "R_o": (0.01, 0.05),
                "p_o": (5.0, 15.0)}
output_names = ["pressure"]

simulator = FlowProblem(
    param_ranges=param_ranges,
    output_names=output_names,
)

In [3]:
input_samples = simulator.sample_inputs(10)
input_samples.shape

torch.Size([10, 11])

In [16]:
input_samples

tensor([[1.8760e+00, 3.4523e-01, 9.2727e+02, 3.8372e-03, 5.6852e+00, 1.0582e+01,
         4.5429e+01, 5.7972e-02, 1.9657e-03, 1.4033e-02, 1.4366e+01],
        [8.7721e-01, 3.1255e-01, 3.0077e+02, 5.0834e-03, 1.0669e+01, 8.9293e+00,
         2.1851e+01, 4.5945e-02, 3.0285e-03, 2.0908e-02, 1.3901e+01],
        [1.4444e+00, 2.5302e-01, 2.1589e+02, 8.5628e-04, 1.7947e+01, 1.3747e+01,
         4.2066e+01, 2.5816e-02, 1.2443e-03, 3.5126e-02, 5.9070e+00],
        [1.0577e+00, 1.0990e-01, 1.1304e+02, 8.4253e-03, 6.6031e+00, 7.5853e+00,
         3.9947e+01, 6.8437e-02, 4.1531e-03, 4.8094e-02, 8.1970e+00],
        [7.2231e-01, 1.7345e-01, 6.4265e+02, 7.6380e-03, 8.9660e+00, 1.5042e+01,
         3.0596e+01, 9.6958e-02, 1.7046e-03, 4.4914e-02, 9.7769e+00],
        [1.5631e+00, 3.9125e-01, 7.5877e+02, 6.4274e-03, 1.9976e+01, 5.4351e+00,
         5.8771e+01, 4.9377e-02, 2.8647e-03, 3.8124e-02, 1.1483e+01],
        [1.8095e+00, 2.6165e-01, 6.3873e+02, 1.9172e-03, 1.3043e+01, 1.6039e+01,
         4.87

In [13]:
single_output = simulator.forward(
    input_samples[0:1]
)

print(single_output)
single_output.shape

tensor([[17.4555,  8.1120, 17.4094, 16.1027, 17.3178, 23.8527, 17.1823, 31.2461,
         17.0047, 38.1727, 16.7877, 44.5290, 16.5347, 50.2204, 16.2492, 55.1621,
         15.9357, 59.2807, 15.5988, 62.5149]])


torch.Size([1, 20])

In [20]:
n_samples = 500
x = simulator.sample_inputs(n_samples).float()
y = simulator.forward_batch(x).float()

Running simulations: 100%|██████████| 500/500 [03:57<00:00, 2.11sample/s] 


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 28 but got size 36 for tensor number 1 in the list.

In [22]:
projectile = Projectile()

# sample from a simulation
n_samples = 500
x = projectile.sample_inputs(n_samples).float()
y = projectile.forward_batch(x).float()

x.shape, y.shape

Running simulations: 100%|██████████| 500/500 [00:00<00:00, 1.17ksample/s]


(torch.Size([500, 2]), torch.Size([500, 1]))

In [23]:
x.shape, y.shape

(torch.Size([500, 2]), torch.Size([500, 1]))

In [24]:
ae = AutoEmulate(x, y, models=[MLP])

Comparing models: 100%|██████████| 1.00/1.00 [01:03<00:00, 63.2s/model]


In [25]:
best = ae.best_result()
model = best.model

In [27]:
problem = {
    'num_vars': simulator.in_dim,
    'names': simulator.param_names,
    'bounds': simulator.param_bounds
    }
sa = SensitivityAnalysis(model, problem=problem)

In [28]:
df = sa.run("sobol")

RuntimeError: The size of tensor a (11) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:
sa.plot(df)